# 1. Indentifying cross-validated feedback history RSA clusters

## Goal

Since we conducted feedback history RSA on the Discovery and Validation group separately, we first need to identify robust or "cross-validated" RSA clusters that encode feedback history model RDMs on both subgroups.

## Details

In [1]:
# Imports
import os
import shutil
from pathlib import Path
from typing import Literal

from nipype.interfaces import afni

In [2]:
# Data paths
discovery_first_level_output_path = Path(
    "[path-to-discovery-bids-dataset]/derivatives/first-level/"  # Please update accordingly
)
validation_first_level_output_path = Path(
    "[path-to-validation-bids-dataset]/derivatives/first-level"  # Please update accordingly
)

# Set cluster output path with respect to the git root
GIT_ROOT_LINES = !git rev-parse --show-toplevel
GIT_ROOT = Path(GIT_ROOT_LINES[0])
cluster_output_path = (
    GIT_ROOT / "second-level" / "output" / "cross_validated_cluster_mask"
)

# Experimental specific constants
SEARCHLIGHT_RADIUS = 3
BLUR_KERNEL_WIDTH = 6

In [3]:
# Function to identify the cross-validated RSA cluster mask for feedback history model (i.e., Recent-2 Trial or Recent-3 Trial)
def identify_cross_validated_cluster_mask(
    model: Literal["recent_2_trial", "recent_3_trial"],
    discovery_path=discovery_first_level_output_path,
    validation_path=validation_first_level_output_path,
    output_path=cluster_output_path,
):
    # Initialize the model cluster output directory
    model_output_path = output_path / model
    os.makedirs(model_output_path, exist_ok=True)

    # Copy each corrected cluster mask to the cluster output directory
    cluster_mask_name = f"feedback_rsa_ttest_{model}_rad{SEARCHLIGHT_RADIUS}_blur{BLUR_KERNEL_WIDTH}_cluster_mask.nii"
    discovery_cluster_mask_name = (
        f"discovery_{model}_cluster_mask.nii"  # Rename due to the name duplication
    )
    discovery_cluster_mask_path = (
        discovery_path
        / "stat"
        / "multivariate"
        / "feedback_model"
        / "ttest"
        / model
        / cluster_mask_name
    )
    assert discovery_cluster_mask_path.exists()
    shutil.copy(
        discovery_cluster_mask_path, model_output_path / discovery_cluster_mask_name
    )

    validation_cluster_mask_name = f"validation_{model}_cluster_mask.nii"
    validation_cluster_mask_path = (
        validation_path
        / "stat"
        / "multivariate"
        / "feedback_model"
        / "ttest"
        / model
        / cluster_mask_name
    )
    assert validation_cluster_mask_path.exists()
    shutil.copy(
        validation_cluster_mask_path, model_output_path / validation_cluster_mask_name
    )

    # Overlap two cluster mask using 3dcalc
    cross_validated_mask_name = f"{model}_cross_validated_cluster_mask.nii"
    os.chdir(model_output_path)
    calc = afni.Calc()
    calc.inputs.in_file_a = discovery_cluster_mask_name
    calc.inputs.in_file_b = validation_cluster_mask_name
    calc.inputs.expr = "and(a, b)"
    calc.inputs.out_file = cross_validated_mask_name
    calc.inputs.args = "-overwrite"
    calc.run()

    # Clean-up copied masks
    (model_output_path / discovery_cluster_mask_name).unlink(missing_ok=True)
    (model_output_path / validation_cluster_mask_name).unlink(missing_ok=True)

    # Return the cross-validated cluster mask
    return model_output_path / cross_validated_mask_name

In [4]:
# Recent-2 Trial cross-validated cluster mask
recent_2_trial_cross_validated_cluster_mask_path = (
    identify_cross_validated_cluster_mask(model="recent_2_trial")
)

240904-18:27:36,426 nipype.interface INFO:
	 stderr 2024-09-04T18:27:36.426050:++ 3dcalc: AFNI version=AFNI_24.1.19 (Jun 14 2024) [64-bit]
240904-18:27:36,427 nipype.interface INFO:
	 stderr 2024-09-04T18:27:36.427263:++ Authored by: A cast of thousands
240904-18:27:36,472 nipype.interface INFO:
	 stderr 2024-09-04T18:27:36.472917:++ Output dataset ./recent_2_trial_cross_validated_cluster_mask.nii


In [5]:
# Recent-3 Trial cross-validated cluster mask
recent_3_trial_cross_validated_cluster_mask_path = (
    identify_cross_validated_cluster_mask(model="recent_3_trial")
)

240904-18:27:36,538 nipype.interface INFO:
	 stderr 2024-09-04T18:27:36.538696:++ 3dcalc: AFNI version=AFNI_24.1.19 (Jun 14 2024) [64-bit]
240904-18:27:36,539 nipype.interface INFO:
	 stderr 2024-09-04T18:27:36.538696:++ Authored by: A cast of thousands
240904-18:27:36,578 nipype.interface INFO:
	 stderr 2024-09-04T18:27:36.578450:++ Output dataset ./recent_3_trial_cross_validated_cluster_mask.nii


In [6]:
# Compute feedback history cluster mask (i.e., Recent-2 Trial cross-validated mask & Recent-3 Trial cross-validated mask)
recent_2_recent_3_overlap_output_path = (
    cluster_output_path / "recent_2_and_recent_3_trial"
)
os.makedirs(recent_2_recent_3_overlap_output_path, exist_ok=True)

# Copy each cross-validated mask
recent_2_mask_name = recent_2_trial_cross_validated_cluster_mask_path.name
recent_3_mask_name = recent_3_trial_cross_validated_cluster_mask_path.name
shutil.copy(
    recent_2_trial_cross_validated_cluster_mask_path,
    recent_2_recent_3_overlap_output_path,
)
shutil.copy(
    recent_3_trial_cross_validated_cluster_mask_path,
    recent_2_recent_3_overlap_output_path,
)

# Overlap two cluster mask using 3dcalc
recent_2_recent_3_overlap_mask_name = "recent_2_recent_3_trial_overlap_cluster_mask.nii"
os.chdir(recent_2_recent_3_overlap_output_path)
calc = afni.Calc()
calc.inputs.in_file_a = recent_2_mask_name
calc.inputs.in_file_b = recent_3_mask_name
calc.inputs.expr = "and(a, b)"
calc.inputs.out_file = recent_2_recent_3_overlap_mask_name
calc.inputs.args = "-overwrite"
calc.run()

# Clean-up copied masks
(recent_2_recent_3_overlap_output_path / recent_2_mask_name).unlink(missing_ok=True)
(recent_2_recent_3_overlap_output_path / recent_3_mask_name).unlink(missing_ok=True)

240904-18:27:36,695 nipype.interface INFO:
	 stderr 2024-09-04T18:27:36.695633:++ 3dcalc: AFNI version=AFNI_24.1.19 (Jun 14 2024) [64-bit]
240904-18:27:36,696 nipype.interface INFO:
	 stderr 2024-09-04T18:27:36.695633:++ Authored by: A cast of thousands
240904-18:27:36,737 nipype.interface INFO:
	 stderr 2024-09-04T18:27:36.737777:++ Output dataset ./recent_2_recent_3_trial_overlap_cluster_mask.nii


## Notes

- For each cross-validated or overlapped cluster mask (e.g., `recent_3_trial_cross_validated_cluster_mask.nii`), we identified clusters whose cluster extents are equal or greater than 20 voxels. Please check the cluster mask file(s) with the `_vox20` suffix in the `second-level/output/cross_validated_cluster_mask/` directory.
    - For the Recent-2 Trial model, all cross-validated clusters were over 20 voxels. So no `*_vox20.nii` file in the directory.

- For the Recent-2 Trial & Recent-3 Trial cluster mask, we named the identified clusters as **feedback history clusters**, which encode both feedback history models. We used these cluster masks in the second-level RSA. Please check our **middle orbital gyrus (MiOG)** and **inferior frontal gyrus (IFG)** cluster masks in the directory.